### MQTT + Wifi-Connection + HTML-Requests/APIs

**Wifi auf dem ESP**

*Erstellen Sie eine Internetverbindung auf dem Mikrocontroller*
- Erstellen Sie eine WLAN-Instanz --> nic = network.WLAN(network.STA_IF)
- Scannen Sie nach möglichen Verbindungen und geben Sie diese aus
  -  nic.active(True)
  -  nic.scan() 
- Erstellen Sie eine Verbindung zu einem Netzwerk
    - nic.connect(ssid,pw)
    - sleep_ms(4000) (Verbindungsaufbau braucht je nach Mikrocontroller/Router etwas Zeit
- Geben Sie die Verbindungsdaten aus
  - nic.ifconfig()


In [ ]:
import network
from utime import sleep_ms

print('check possible connections around this device')
nic = network.WLAN(network.STA_IF)
# Activate interface if deactivated
if not nic.active():
    nic.active(True)
print('Interface activated: ' + str(nic.active()))
# Scan for available networks
wlan_list = nic.scan()
# Print results
for idx, wlan in enumerate(wlan_list):
    print('Network ' + str(idx) + ':')
    print('SSID: ' + wlan[0].decode())
    print('RSSI: ' + str(wlan[3]) + 'dBm')
ssid = ''
pw = ''
# Connect to extising WLAN
nic.connect(ssid, pw)
sleep_ms(4000) #1000 hat bei iphone nicht funktioniert
if nic.isconnected():
    print('Connected to ' + ssid + '.')
    print('config tuple: IP, subnet, gateway, DNS')
    print(nic.ifconfig())
    print('süüüüpeeer!')
    
else:
    print('Connection to ' + ssid + ' could not be established.')
# Receive and print network configuration
nic.disconnect()
print(nic.isconnected())



**Umwandlung in Funktionen**

- *Schreiben Sie eine Funktion checkWifi, die prüft, ob eine Verbindung besteht*
- *Schreiben Sie eine Funktion connectToWifi(ssid,pw), die eine Wlan-Verbindung aufbaut, sofern noch keine Verbindung besteht*

In [ ]:
import network
from utime import sleep_ms
import urequests

def check_wifi_connections():
    print('check possible connections around this device')
    nic = network.WLAN(network.STA_IF)
    # Activate interface if deactivated
    if not nic.active():
        nic.active(True)
    print('Interface activated: ' + str(nic.active()))
    # Scan for available networks
    wlan_list = nic.scan()
    # Print results
    for idx, wlan in enumerate(wlan_list):
        print('Network ' + str(idx) + ':')
        print('SSID: ' + wlan[0].decode())
        print('RSSI: ' + str(wlan[3]) + 'dBm')
    
def connect_to_wifi(ssid,pw):
    
    #print ('lade wifi connect method')
    nic = network.WLAN(network.STA_IF)
    # Activate interface if deactivated
    if not nic.active():
        nic.active(True)
    print('Interface activated: ' + str(nic.active()))
    if nic.isconnected():
        print('verbunden')
        #print("bereits verbunden mit ip "+nic.ifconfig())
    else:
        # Connect to extising WLAN

        nic.connect(ssid, pw)
        sleep_ms(4000) #1000 did not work
        if nic.isconnected():
            print('Connected to ' + ssid + '.')
            print('config tuple: IP, subnet, gateway, DNS')
            print(nic.ifconfig())
            
        else:
            print('Connection to ' + ssid + ' could not be established.')

**http Test**
*Um zu testen, ob Sie nicht nur im Wlan, sondern auch im Internet sind, nutzen Sie die http request get-Methode, um Daten von einer Homepage oder einer API abzufragen. Nutzen Sie dazu eine API und Homepage Ihrer Wahl*
- Nutzen Sie zunächst die vorhandenen Funktionen, um einen WLan-Verbindung aufzubauen.
- Greifen Sie anschließend auf eine Homepage und eine API Ihrer Wahl zu

Wenn Sie hierzu den Mikrocontroller nutzen -> 
- import urequests
- response=urequests.get(url)...

Auf dem Laptop, sprich mit Python
- import requests
- response=requests.get(url)...

(eine ausführlichere Übung zu APIs folgt im Mai)

In [ ]:
# HTTP API-Request 
# Witzeseite
import urequests
url = "https://v2.jokeapi.dev/joke/Any?type=single"

res = urequests.get(url, headers={"User-Agent": "ESP8266"})
print("Status:", res.status_code)
print("Antwort:", res.text)

data = res.json()
print(data["joke"])
res.close()

# einfacher http-test
try:
    print("verbindung aufbauen mit http://example.com ...")
    response = urequests.get("http://example.com")
    print("Response status: (200 ok, 404 not found...", response.status_code)
    print("snippet von website als beweis", response.text[:100])  # Print first 100 characters
    response.close()
except Exception as e:
        print("Failed to reach the internet:", e)

# https Test, wenn kein Fehler, wird https untestützt (nicht auf allen ESP8266 möglich)
try:
    r = urequests.get("https://www.google.com")
    print("Status:", r.status_code)
    r.close()
except Exception as e:
    print("Fehler:", e)

**Einrichtung MQTT**

- Establish wifi connection
- Mosquitto mqtt auf Server (sprich Laptop) installieren
- Pfad auf mosquitto in den Systemvariablen definieren oder bei den command-line-Befehlen absolute Pfade nutzen
- Umqtt.simple auf mikrocontroller installieren (geht nicht über pip, weil gibt kein pip für micropython)	
- Download umqtt.simple.py (or clone whole micropython from github) --> https://github.com/micropython/micropython-lib/blob/master/micropython/umqtt.simple/umqtt/simple.py
- Upload to microcontroller: 
    - mpremote connect COM3 fs cp simple.py :umqtt/simple.py
    - Oder Kopie der Datei auf den Mikrocontroller mit thonny
- Troubleshooting siehe Folien


**Start MQTT**

*Testen Sie am Laptop, ob und wie MQTT bei Ihnen funktioniert, indem Sie Mosquitto starten, einen broker öffnen und einen subscriber öffnen*

(wenn möglich in der Admin-CMD) den folgenden Befehl eingeben, um Mosquitto-MQTT zu starten:
- mosquitto -c "C:\Program Files\mosquitto\mosquitto_copy.conf" -v (-> Pfad soll auf IHRE .config-Datei verweisen)
- in zweiter Konsole: mosquitto_sub -h localhost -t "temperature"
- in dritter Konsole: mosquitto_pub -h localhost -t "temperature" -m "testtesttest"

mosquitto_sub ist der subscriber, pub ist der publisher. Localhost bedeutet, der broker ist local. Das Thema ist "temperature"
Lassen Sie die subscriber-Console offen, diese hört jetzt zu und wartet ;-)
 --> Schicken Sie sich einige Testnachrichten

**mqtt receive messages on esp**

*Erstellen Sie jetzt nochmals die Konsolen mit publishern und subscribern, allerdings dieses Mal nicht mehr als localhost, sondern versehen mit der IP4-Adresse ihres Laptops innerhalb des Netzwerks*

(1:1 Ersetzen von localhost durch die Adresse 192...)

*Erstellen Sie anschließend auf dem Mikrocontroller ein Skript, das ebenfalls subscribed*
- nutzen Sie dazu erneut die wifi-Funktion
- erstellen Sie einen Client: client = MQTTClient("esp8266_sub", "172.20.10.13")  # PC IP
- erstellen Sie eine Callback-Function
- Ein Callback bei MQTT ist eine Funktion, die automatisch aufgerufen wird, wenn ein bestimmtes Ereignis eintritt – zum Beispiel, wenn:
    - eine Nachricht empfangen wurde (on_message)
    - die Verbindung zum Broker erfolgreich hergestellt wurde (on_connect)
    - die Verbindung abbricht (on_disconnect)
    - Nehmen Sie eine Verbindung zum Client auf --> client.connect()
    - Subscriben Sie zum Thema, das auch auf der Konsole läuft, z.B. Temperatur: client.subscribe(b"temperature_vong_laptop")
    - warten Sie in einer Schleife auf Nachrichten: client.wait_msg()
      

In [ ]:
from network import WLAN, STA_IF
from utime import sleep_ms
from umqtt.simple import MQTTClient
import connect_to_wifi as cw # eigenes ausgelagertes wifi skript

#cw.check_wifi_connections()
print('through connection overview')
# if connection established, this can be commented out
cw.connect_to_wifi(ssid,pw)

# Callback when a message is received
def sub_cb(topic, msg):
    print(f"Received: Topic={topic}, Message={msg}")

# Setup MQTT client and connect to broker
client = MQTTClient("esp8266_sub", "172.20.10.13")  # IP des Rechners (individuell und auch nach Neustart oder je nach Netzwerk anders)
client.set_callback(sub_cb)
client.connect()

# Subscribe to topic
client.subscribe(b"temperature_laptop")
print("Subscribed to temperature_laptop")
# Wait for messages
try:
    while True:
        client.wait_msg()  # Blocking wait
        # use client.check_msg() if you want non-blocking instead
except KeyboardInterrupt:
    print("Stopped by user")
    client.disconnect()


**MQTT Client auf dem Mikrocontroller**

Stand jetzt sollte Mosquitto auf dem Laptop (=Broker) laufen, Laptop und Mikrocontroller sollten im gleichen Netzwerk sein und ein erster Test sollte stattgefunden haben.

- *Erstellen Sie einen Client auf dem Mikrocontroller, der Dummy-Werte erzeugt und published*
- *Erstellen Sie einen Subscriber auf Ihrem Laptop mithilfe von Paho mqtt, der zum gleichen Thema subscribed und die Nachrichten abruft*

In [ ]:
# Mikrocontroller - mqtt client

from network import WLAN, STA_IF
from utime import sleep_ms
from umqtt.simple import MQTTClient
import connect_to_wifi as cw # eigenes ausgelagertes wifi skript

ssid = ''
pw = ''

cw.check_wifi_connections()
# connect to wifi
cw.connect_to_wifi(ssid,pw)

brokerAddresse="172.20.10.13"# cmd ipconfig eingeben
thema = b'temperatur_vong_esp'
clientName = 'esp8266'
print("MQTT-Broker:", brokerAddresse)
mqtt_client = MQTTClient(clientName, brokerAddresse)  # IP vom Broker (PC)
mqtt_client.connect()

print('through connect')

# Schleife, die in der theorie verschiedene Messwerte ausgeben durchlaufen und publishen könnte
for i in range(3):
    temp1=i
    message1=(f"Temperatur am Sensor: {temp1}")
    # client published zu einem bestimmten Thema und mit einer bestimmten Nachricht
    mqtt_client.publish(thema, str(message1))
    time.sleep(.5)
print('end of file')


on_connect(client, userdata, flags, rc) ist eine Callback-function (sh. oben)

- client: Das Client-Objekt selbst (damit kann man z. B. client.subscribe(...) aufrufen).
- userdata: Beliebige benutzerdefinierte Daten (wird beim Erstellen des Clients festgelegt; meist None).
- flags: Verbindungsflags, z. B. ob es sich um eine Session-Wiederherstellung handelt (wird selten gebraucht).
- rc: Return Code – zeigt an, ob die Verbindung erfolgreich war (siehe unten im Aufruf).

In [ ]:
# subscriber auf PC

import paho.mqtt.client as mqtt

# Callback when connected to broker
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe("temperatur_vong_esp")

# Callback when a message is received
def on_message(client, userdata, msg):
    print(f"[{msg.topic}] {msg.payload.decode()}")

# MQTT setup
client = mqtt.Client()

client.on_connect = on_connect
client.on_message = on_message

# Connect to PC broker (same IP the ESP uses!)
client.connect("172.20.10.13", 1883, 60) 
# 1883 ist der offene Port
# 60 bedeutet, dass der client maximal 60sec verstreichen lässt, bevor er nervös wird  und ein ping gesendet wird. gibt es dann keine Reaktion, trennt der Client die Verbindung

# Endlosloop, Client hört zu
client.loop_forever()

**MQTT + Temperature Work-Flow**

*Der Client auf Ihrem lokalen PC ruft Nachrichten ab und Sie wissen jetzt, wie Sie über MQTT kommunizieren. Jetzt erstellen Sie einen Workflow*

*file 1 - Mikrocontroller*
1. Temperatur wird in einer separaten Funktion über einen Temperatursensor bestimmt
2. Temperaturfunktion wird aufgerufen und in einer Variablen gespeichert.
3. Anschließend wird in einem Loop die Temperatur in ein Dictionary geschrieben und anschließend in ein json-File (json.dumps(tempDict))
4. das wird dann über MQTT verschickt. Verschicken Sie ebenfalls ein Stop-Signal, sodass der Client weiß, wann die letzte Nachricht kommt (stop=False/True)

*file 2 - python/Laptop*

5. in einem zweiten Client auf Ihrem Laptop soll die Nachricht empfangen werden, das Json in ein dict umgewandelt werden --> data = json.loads(msg.payload)
6. anschließend wird das dict in ein pandas Dataframe geschrieben, das nach und nach mit den eingehenden Nachrichten aufgefüllt wird
7. wird eine Nachricht erhalten, die stop=True enthält, wird die Verbindung unterbrochen und das Skript läuft weiter
8. Stellen Sie sicher, dass das DataFrame auch noch verfügbar ist, nachdem die Verbindung abgebrochen ist. Dies ist möglich, indem Sie den DataFrame als globale Variable außerhalb der Client-Funktionen definieren und innerhalb der Funktionen mit "global" markieren
9. jetzt haben Sie die Möglichkeit, die erhaltenen Daten zu plotten, in eine Datenbank zu schreiben etc.!
10. dieser workflow ist tricky, enthält aber dafür bereits 50% Ihres Miniprojekts. Er wird daher in der VL erläutert damit es Ihnen ermöglicht wird, diesen zunächst ohne Lösung zu erarbeiten :-)